# Preparation

## Import Library

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from zipfile import ZipFile

## Import Data

In [2]:
with ZipFile('fake-and-real-news-dataset.zip') as raw_zip:
    raw_zip.extractall(path='datasets')
    
    file_name = raw_zip.namelist()
    
    raw_data = []
    
    for file in file_name:
        raw_data.append(pd.read_csv(f'datasets/{file}'))
        
    print(file_name)

['Fake.csv', 'True.csv']


In [9]:
fake_news = raw_data[0]
real_news = raw_data[1]

## Sampling Data

In [34]:
from sklearn.utils.random import sample_without_replacement

fake_news_index_sampling = sample_without_replacement(fake_news.shape[0], 5000, random_state=0)
real_news_index_sampling = sample_without_replacement(real_news.shape[0], 5000, random_state=0)

fake_news_sampled = fake_news.loc[fake_news_index_sampling]
real_news_sampled = real_news.loc[real_news_index_sampling]

## Add Label

In [40]:
fake_news_sampled['fake'] = 1
real_news_sampled['fake'] = 0

## Concat Data

In [42]:
full_data = pd.concat([fake_news_sampled, real_news_sampled], axis=0)

full_data.head()

,title,text,subject,date,fake
9997,Heaven’s Gatekeeper? Jesse Jackson Proclaims T...,Jesse Jackson thinks he s Heaven s gatekeeper ...,politics,"Sep 2, 2017",1
8035,Here’s What The World’s Top Economist Is Sayi...,"Thomas Piketty, the French economist who start...",News,"February 17, 2016",1
644,Five Times Five-Time Draft Dodger Donald Trum...,"Instead of honoring Purple Heart Day, Trump be...",News,"August 7, 2017",1
5105,Hillary Clinton: Trump ‘Crossed The Line’ Wit...,Hillary Clinton took a swing back at Donald Tr...,News,"August 10, 2016",1
21573,FATHER OF HIGH SCHOOL FOOTBALL STAR SHOT AND K...,They shot him in the head with a 45. They fo...,left-news,"Jul 11, 2015",1


# Exploratory Data Analysis and Data CLeaning

## News Title